In [1]:
import torch 
import matplotlib.pyplot as plt 
import torchvision
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from module import UNet
import torch.nn as nn
from diffusion import Diffusion
from tqdm import tqdm

d:\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transforms=transforms.Compose([
    transforms.Resize(80),
    transforms.RandomResizedCrop(10,scale=(0.8,1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
dataset=torchvision.datasets.CIFAR10(root='./',download=False,transform=transforms)
dataloader=DataLoader(dataset,10,shuffle=True)

In [4]:
def train():
    device='cpu'
    model=UNet().to(device)
    optimizer=torch.optim.AdamW(model.parameters(),lr=1e-4)
    mse=nn.MSELoss()
    diffusion=Diffusion(img_size=64)
    l=len(dataloader)
    for epoch in range(2):
        print(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        for i, (images, _) in enumerate(pbar):
            images = images.to(device)
            t = diffusion.sample_timesteps(images.shape[0])
            x_t, noise = diffusion.noise_image(images, t)
            predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
        torch.save(model.state_dict(), 'weight.pt')